# Linear Regression

This is a simple linear regression model trained using TensorFlow's simplest optimizer called **gradient descent**.

In [1]:
import tensorflow as tf

print("Running on TensorFlow %s" % tf.__version__)

# REF: http://stackoverflow.com/questions/39356714/using-tensorboard-with-jupyter-notebooks
tf.reset_default_graph()

sess = tf.InteractiveSession()

Running on TensorFlow 1.0.1


Trainable parameters in a graph are set using **Variables**. These are what make the model trainable, allowing it to get new outputs using the same inputs.

They have a type and initial value:

In [2]:
# Model parameters
with tf.name_scope('weights'):
    W = tf.Variable([.3], tf.float32)
with tf.name_scope('bias'):
    b = tf.Variable([-.3], tf.float32)

For the graph to accept external inputs, we must define **placeholders**. They are essentially a promisse to provide value later:

In [3]:
# Model input
x = tf.placeholder(tf.float32)

Define an operation on our input and trainable parameters:

In [4]:
linear_model = W * x + b

Let's now define our training data:

In [5]:
# training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]

In TensorFlow, constants are initialized when they are defined, and their value can never change. By contrast, variables are not initialized automatically. To initialize all the variables in a TensorFlow program, we must explicitly call `global_variables_initializer()`:

In [6]:
init = tf.global_variables_initializer().run()

At this point, we already can evaluate our linear model giving it some values for its `x` placeholder. It can even be our actual training data, however, we don't know how good it is...

In [7]:
print(sess.run(linear_model, {x: [5, 1, 4, 9]}))
print(sess.run(linear_model, {x: x_train}))

[ 1.20000005  0.          0.90000004  2.4000001 ]
[ 0.          0.30000001  0.60000002  0.90000004]


To evaluate a model on training data, we need a `y` placeholder to provide the desired values and a function that tells us how it is performing.
A model can either have an *utility function* or a *cost function*, it depends on whether we want to define **how good** it is perming, or **how bad**, respectively.

Here we are defining our *cost function*, also known as *loss function*. It is a function of our linear model and the ground truth:

In [8]:
# Model output
y = tf.placeholder(tf.float32)

# loss
loss = tf.reduce_sum(tf.square(linear_model - y))  # sum of the squares
tf.summary.scalar('loss', loss)

<tf.Tensor 'loss:0' shape=() dtype=string>

Our gradient descent optimizer will have a *learning rate* of `0.001`. This type of optimizer of a loss function modifies each variable according to the magnitude of the derivative of loss with respect to that variable:

In [9]:
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

Train the optimizer `1000` times giving it the training data:

In [10]:
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter('./summaries', sess.graph)

init = tf.global_variables_initializer().run()

# training loop
for i in range(1000):
    summary, _ = sess.run([merged, train], {x: x_train, y: y_train})
    train_writer.add_summary(summary, i)

train_writer.close()

Results in the final model parameters:

In [11]:
print(sess.run([W, b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


To evaluate this model we must invoke `run()` to make TensorFlow run **enough of the computational graph** to evaluate `W`, `b` and `loss`:

In [12]:
# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
print("W: %s b: %s loss: %s" % (curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


Our loss decreases with every training step as expected:
![](images/trainable_linear_regression_loss.png)

From **TensorBoard**, here is our graph:
![](images/trainable_linear_regression_graph_v2.png)